In [ ]:
#!/usr/bin/env python
from __future__ import print_function, division
import os
import time
import numpy as np
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm
from os.path import join
from utils.ssim import SSIM, MSSSIM
from parameters import Parameters
from utils.utils import adjust_learning_rate
import matplotlib.pyplot as plt

from utils.dataset import create_my_data_3ch, BasicDataset
from loupe.models import loupe_3ch

from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import normalized_root_mse
from skimage.metrics import structural_similarity
from skimage.metrics import mean_squared_error

seed_num = 42
torch.manual_seed(seed_num)
torch.cuda.manual_seed_all(seed_num)
np.random.seed(seed_num)
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
params = Parameters()
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
params.epochs = 5 #40
params.batch_size = 32 # 128
params.lr = 0.01 # 0.01
params.sparsity = 0.125

In [ ]:
# Project config
model_name = params.model_name
print("model_name:",model_name)
num_epoch = int(params.epochs)
batch_size = int(params.batch_size)
ssimCriterion = SSIM()
msssimCriterion = MSSSIM()

In [ ]:
# Configure directory info
run_name = model_name+"_bs_"+str(params.batch_size) + "_ep_"+str(params.epochs) + "_lr_" + str(params.lr)
save_dir = join(params.save_weights, run_name)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
global_step = 0

In [ ]:
trainx, trainy, valx, valy, train_mean, train_std = create_my_data_3ch(params)

In [ ]:
n_train = int(len(trainx))
n_val = int(len(valx))
dataset_train = BasicDataset(trainx, trainy)
dataset_val = BasicDataset(valx, valy)
train_loader = DataLoader(dataset_train, batch_size=params.batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset_val, batch_size=params.batch_size, shuffle=False, num_workers=0)

In [ ]:
n_train = int(len(trainx))
n_val = int(len(valx))
dataset_train = BasicDataset(trainx, trainy)
dataset_val = BasicDataset(valx, valy)
train_loader = DataLoader(dataset_train, batch_size=params.batch_size, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset_val, batch_size=params.batch_size, shuffle=False, num_workers=0)

In [ ]:
rec_net = loupe_3ch(6,6,params.sparsity)
#density_compensation = torch.from_numpy(density_compensation).cuda().requires_grad_(True)
criterion = torch.nn.L1Loss().cuda()
optimizer = optim.Adam(rec_net.parameters(), lr=float(params.lr), betas=(0.5, 0.999))
if cuda:
    rec_net = rec_net.cuda()
rec_net.train()
best_loss = 9999
best_model_name = ''

In [ ]:
for epoch in range(num_epoch):
    rec_net.train()
    optimizer = adjust_learning_rate(epoch, optimizer)
    count = 1
    t_loss = 0
    n_loss = 0
    r_loss = 0
    with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{num_epoch}', unit='img') as pbar:
        for batch in train_loader:
            optimizer.zero_grad()

            img_un = batch['img_un'].cuda().float() #[b 6 96 96] n c w h 
            
            img_full_real = batch["img_full_real"].cuda().float()

            img_full_real_input = img_full_real#.repeat(1,3,1,1)
            
            out_pred, out_mask, out_prob = rec_net(img_un)

            pred_loss1= criterion(out_pred[:,0,:,:],img_full_real_input[:,0,:,:])
            pred_loss2= criterion(out_pred[:,1,:,:],img_full_real_input[:,1,:,:])
            pred_loss3= criterion(out_pred[:,2,:,:],img_full_real_input[:,2,:,:])

            mask_loss1=  criterion(out_prob[:,0,:,:],out_prob[:,1,:,:])
            mask_loss2=  criterion(out_prob[:,1,:,:],out_prob[:,2,:,:])
            mask_loss3=  criterion(out_prob[:,2,:,:],out_prob[:,0,:,:])
            
            n_loss = (pred_loss1+pred_loss2+pred_loss3)/3
            r_loss = 0.001 * (mask_loss3+mask_loss1+mask_loss2)
            
            loss = 0
            loss = n_loss - r_loss

            #t_loss = t_loss + loss.item()
            count += 1
            #writer.add_scalar('train/Loss', loss.item(), global_step)
            pbar.set_postfix({'loss': n_loss, 'reg':r_loss})
  
            loss.backward()
            #n_loss.backward()

            optimizer.step()
            pbar.update(img_un.shape[0])
            global_step += 1
    if epoch % 10 == 0:
        pred_img = out_pred.detach().cpu().numpy()
        #print("pred_img.shape",pred_img.shape)
        plt.figure()
        plt.imshow(pred_img[0,0,:,:], cmap='gray')
        #plt.savefig(str(epoch)+"_img.png") 

In [ ]:
rec_net.eval()
psnr_total = 0
nmse_total = 0
rmse_total = 0
ssim_total = 0
num = 0
with tqdm(total=n_val, desc=f'Epoch {epoch + 1}/{num_epoch}', unit='img') as pbar:
    for batch in val_loader:
        img_un = batch['img_un'].cuda().float()
        img_full_real = batch["img_full_real"].cuda().float()
        img_full_real_input = img_full_real#.repeat(1,3,1,1)
        out_pred, out_mask, out_prob = rec_net(img_un)

        pred_img = out_pred.detach().cpu().numpy()
        real_img = img_full_real_input.detach().cpu().numpy()

        for i in range(pred_img.shape[0]):
            psnr_total += peak_signal_noise_ratio(real_img[i][0],pred_img[i][0])
            psnr_total += peak_signal_noise_ratio(real_img[i][1],pred_img[i][1])
            psnr_total += peak_signal_noise_ratio(real_img[i][2],pred_img[i][2])

            nmse_total += normalized_root_mse(real_img[i][0],pred_img[i][0])
            nmse_total += normalized_root_mse(real_img[i][1],pred_img[i][1])
            nmse_total += normalized_root_mse(real_img[i][2],pred_img[i][2])

            rmse_total += np.sqrt(mean_squared_error(real_img[i][0],pred_img[i][0]))
            rmse_total += np.sqrt(mean_squared_error(real_img[i][1],pred_img[i][1]))
            rmse_total += np.sqrt(mean_squared_error(real_img[i][2],pred_img[i][2]))

            ssim_total += structural_similarity(real_img[i][0],pred_img[i][0],data_range=1)
            ssim_total += structural_similarity(real_img[i][1],pred_img[i][1],data_range=1)
            ssim_total += structural_similarity(real_img[i][2],pred_img[i][2],data_range=1)
            num +=1

        pbar.update(img_un.shape[0])

psnr_mean = psnr_total / n_val / 3
nmse_mean = nmse_total / n_val / 3
rmse_mean = rmse_total / n_val / 3
ssim_mean = ssim_total / n_val / 3

print("BASELINE PSNR:%s SSIM:%s NMSE:%s RMSE:%s" % (psnr_mean,ssim_mean,nmse_mean,rmse_mean))

In [ ]:
prob_mask = out_mask.detach().cpu().numpy()
prob_mask = prob_mask[0]
pred_img = out_pred.detach().cpu().numpy()
pred_img = pred_img[0]
real_img = img_full_real_input.detach().cpu().numpy()
real_img = real_img[0]

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(12,4))
plt.suptitle("LOUPE, 8x Acceleration",size='xx-large')
for i in range(3):
    axes[i].imshow(np.round(prob_mask[i]), cmap='gray', interpolation='none')
    axes[i].axis('off')

In [ ]:
np.mean(prob_mask[2])

In [ ]:
fig, ax = plt.subplots(3,3,figsize=(12,12))
plt.suptitle("LOUPE, 4x Acceleration",size='xx-large')
ax[0,0].imshow(pred_img[0],cmap='gray')
ax[0,0].set_title('Output')
ax[0,1].imshow(real_img[0],cmap='gray')
ax[0,1].set_title('Ground Truth')
ax[0,2].imshow(np.abs(real_img[0]-pred_img[0]),cmap='gray')
ax[0,2].set_title('Difference')

for i in range(3):
    ax[i,0].imshow(pred_img[i],cmap='gray')
    ax[i,1].imshow(real_img[i],cmap='gray')
    ax[i,2].imshow(np.abs(real_img[i]-pred_img[i]),cmap='jet')
    ax[i,0].axis('off')
    ax[i,1].axis('off')
    ax[i,2].axis('off')

In [ ]:
np.save('3 channel, slope=200, sample_slope=200, 40iters, lr=0.01, sparsity=0.25, lambda=0.001, 8x.npy', [prob_mask,pred_img])

In [ ]:
torch.save(rec_net.state_dict(), '3 channel, slope=200, sample_slope=200, 40iters, lr=0.01, sparsity=0.25, lambda=0.001, 8x')